<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-08 09:09:11--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-08 09:09:11 (132 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-08 09:09:14--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.250.65.80, 142.250.188.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-09-08 09:09:14 (161 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df),len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:1 (real disaster)
Text
 950. If a landslide tumbles down todayI'm on your side

And if a twister sweeps it all away-

YOU'RE ON YOUR OWN BITCH!*runs into distance* 

---

target:0 (not a disaster)
Text
 So the Ahamedis think the Messiah had already come 125 years ago? Where is Armageddon? Where is the Dajaal? Where is Gog &amp; Magog?! 

---

target:1 (real disaster)
Text
 @camilacabello97 Internally and externally screaming 

---

target:0 (not a disaster)
Text
 @engineermataRAI ate mataas kc rating..but they did not think on doing like this they destroy the story along with the ratings 

---

target:0 (not a disaster)
Text
 @AminESPN Mencius tears are worse correct? Takes the explosion n more pain day to day right? 

---



## splitting the data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

In [15]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [16]:
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 IF ANYONE WERE TO HARM THE BOYS THEY WOULD GET TAKEN DOWN IMMEDIATELY NOT BY SECURITY BUT BY THE FANS REAL QUICK       
 Vectorized version:
 [[  47  864   67    5  396    2 1547   64   78   52  806  134 5385   34
    18]]


In [25]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [29]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [33]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Mariah getting thick in the shoulders poor girl.         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00576361,  0.04824756,  0.00281329, ..., -0.03378771,
          0.00199059,  0.01366599],
        [ 0.01065721,  0.00234121, -0.02355003, ...,  0.04914523,
         -0.01999396,  0.02596844],
        [-0.04661259, -0.02170429,  0.02312047, ...,  0.00210383,
          0.03924546, -0.0300487 ],
        ...,
        [ 0.04086875, -0.0158455 ,  0.02702106, ..., -0.0318864 ,
          0.01825793,  0.02417772],
        [ 0.04086875, -0.0158455 ,  0.02702106, ..., -0.0318864 ,
          0.01825793,  0.02417772],
        [ 0.04086875, -0.0158455 ,  0.02702106, ..., -0.0318864 ,
          0.01825793,  0.02417772]]], dtype=float32)>

In [37]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

('Mariah getting thick in the shoulders poor girl.',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-5.7636127e-03,  4.8247565e-02,  2.8132908e-03,  4.7648046e-02,
         3.4992125e-02, -2.4258494e-03,  4.9605060e-02,  1.2423553e-02,
        -7.1201436e-03, -1.7664790e-02, -1.0541081e-03,  1.0770701e-02,
        -2.6972378e-02,  4.4758711e-02,  1.9277301e-02,  1.8797759e-02,
        -2.3206366e-02, -2.7442515e-02,  7.8002214e-03,  1.1983573e-02,
         4.3100979e-02, -9.1938861e-03, -1.6808260e-02,  4.7981966e-02,
        -3.7052810e-02,  3.5790954e-02,  4.1875985e-02, -4.9858939e-02,
        -4.2242017e-02, -1.7906547e-02,  1.5630495e-02,  3.6927734e-02,
         3.8541928e-03, -3.5080314e-02,  8.1706420e-03,  3.0136060e-02,
         4.9851645e-02, -4.0550411e-02,  3.7709627e-02,  3.0684952e-02,
        -3.6086392e-02, -3.0278360e-02,  4.3199968e-02, -3.7118874e-02,
        -2.6558030e-02, -1.7711330e-02,  3.1973731e-02,  3.7193689e-02,
         6.4092763e-03, -4.2560231

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [46]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [47]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [53]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [55]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}
